In [ ]:
import pandas as pd
import os
import sys
from math import log
from math import sqrt
import matplotlib.pyplot as plt 
from collections import defaultdict

In [ ]:
def candlestick(cluster):
    candlestick_cluster = {}
    
    for day in range(len(cluster)):
        
        if(day==0):
            continue
        upper = 100*(cluster[day]['High'] - max(cluster[day]['Open'], cluster[day]['Close']))/cluster[day]['Open']
        lower = 100*(min(cluster[day]['Open'], cluster[day]['Close']) - cluster[day]['Low'])/cluster[day]['Open']
        body = 100*(cluster[day]['Close']- cluster[day]['Open'])/cluster[day]['Close']
        if(cluster[day]['Low'] <= cluster[day-1]['High']):
            gap=0
        else:
            gap=100*(cluster[day]['Low']- cluster[day-1]['High'])/cluster[day]['Low']
        trend = 100*(cluster[day]['Close']- cluster[day-1]['Close'])/cluster[day]['Close']
        if(cluster[day-1]['Low'] <= cluster[day]['Open']):
            difopen=0
        else:
            difopen=100*(cluster[day-1]['Low']- cluster[day]['Open'])/cluster[day-1]['Low']
        if(cluster[day-1]['High'] >= cluster[day]['Close']):
            difclose=0
        else:
            difclose=100*(cluster[day]['Close']- cluster[day-1]['High'])/cluster[day]['Close']
        if(cluster[day]['Close'] <= (cluster[day-1]['Open']+cluster[day-1]['Close'])/2):
            difcentral=0
        else:
            difcentral=100*(cluster[day]['Close'] - (cluster[day-1]['Open']+cluster[day-1]['Close'])/2)/cluster[day]['Close']
        rsi=cluster[day]['RSI']
        
        data={'Upper': upper, 'Lower': lower, 'Body': body, 'Gap': gap, 'Trend':trend, 'Difopen': difopen, 'Difclose': difclose, 'Difcentral': difcentral, 'RSI': rsi}
        candlestick_cluster[day]=data
        
    return candlestick_cluster

In [ ]:
#membership function
def fuzzify_candlestick(data):
    
    fuzzified_candlestick_cluster={}
    
    for day in range(6):
        
        if(day==0):
            continue

        Fuzzy_Lower=''
        Fuzzy_Upper=''
        Fuzzy_Body=''
        Fuzzy_Trend=''
        Fuzzy_Gap=''
        Fuzzy_Difopen=''
        Fuzzy_Difclose=''
        Fuzzy_Difcentral=''
        RSI=''

        #lower
        fuzzy_lower={}
        if(data[day]['Upper']>=0 and data[day]['Upper']<=0.5):
            fuzzy_lower['NULL']=-2*data[day]['Upper']+1
            fuzzy_lower['SHORT']=2*data[day]['Upper']
            fuzzy_lower['MIDDLE']=0
            fuzzy_lower['LONG']=0

            if(max(fuzzy_lower['NULL'], fuzzy_lower['SHORT'])==fuzzy_lower['NULL']):
                Fuzzy_Lower='NULL'
            else:
                Fuzzy_Lower='SHORT'

        elif(data[day]['Upper']>0.5 and data[day]['Upper']<=1.5):
            fuzzy_lower['NULL']=0
            fuzzy_lower['SHORT']=1
            fuzzy_lower['MIDDLE']=0
            fuzzy_lower['LONG']=0

            Fuzzy_Lower='SHORT'

        elif(data[day]['Upper']>1.5 and data[day]['Upper']<=2.5):
            fuzzy_lower['NULL']=0
            fuzzy_lower['SHORT']=-data[day]['Upper']+2.5
            fuzzy_lower['MIDDLE']=data[day]['Upper']-1.5
            fuzzy_lower['LONG']=0

            if(max(fuzzy_lower['MIDDLE'], fuzzy_lower['SHORT'])==fuzzy_lower['MIDDLE']):
                Fuzzy_Lower='MIDDLE'
            else:
                Fuzzy_Lower='SHORT'

        elif(data[day]['Upper']>2.5 and data[day]['Upper']<=3.5):
            fuzzy_lower['NULL']=0
            fuzzy_lower['SHORT']=0
            fuzzy_lower['MIDDLE']=1
            fuzzy_lower['LONG']=0

            Fuzzy_Lower='MIDDLE'

        elif(data[day]['Upper']>3.5 and data[day]['Upper']<=5):
            fuzzy_lower['NULL']=0
            fuzzy_lower['SHORT']=0
            fuzzy_lower['MIDDLE']=(-2*data[day]['Upper']+10)/3
            fuzzy_lower['LONG']=(2*data[day]['Upper']-7)/3

            if(max(fuzzy_lower['MIDDLE'], fuzzy_lower['LONG'])==fuzzy_lower['MIDDLE']):
                Fuzzy_Lower='MIDDLE'
            else:
                Fuzzy_Lower='LONG'

        elif(data[day]['Upper']>5):
            fuzzy_lower['NULL']=0
            fuzzy_lower['SHORT']=0
            fuzzy_lower['MIDDLE']=0
            fuzzy_lower['LONG']=1

            Fuzzy_Lower='LONG'
            
        #upper
        fuzzy_upper={}
        if(data[day]['Upper']>=0 and data[day]['Upper']<=0.5):
            fuzzy_upper['NULL']=-2*data[day]['Upper']+1
            fuzzy_upper['SHORT']=2*data[day]['Upper']
            fuzzy_upper['MIDDLE']=0
            fuzzy_upper['LONG']=0

            if(max(fuzzy_upper['NULL'], fuzzy_upper['SHORT'])==fuzzy_upper['NULL']):
                Fuzzy_Upper='NULL'
            else:
                Fuzzy_Upper='SHORT'

        elif(data[day]['Upper']>0.5 and data[day]['Upper']<=1.5):
            fuzzy_upper['NULL']=0
            fuzzy_upper['SHORT']=1
            fuzzy_upper['MIDDLE']=0
            fuzzy_upper['LONG']=0

            Fuzzy_Upper='SHORT'

        elif(data[day]['Upper']>1.5 and data[day]['Upper']<=2.5):
            fuzzy_upper['NULL']=0
            fuzzy_upper['SHORT']=-data[day]['Upper']+2.5
            fuzzy_upper['MIDDLE']=data[day]['Upper']-1.5
            fuzzy_upper['LONG']=0

            if(max(fuzzy_upper['SHORT'], fuzzy_upper['MIDDLE'])==fuzzy_upper['MIDDLE']):
                Fuzzy_Upper='MIDDLE'
            else:
                Fuzzy_Upper='SHORT'

        elif(data[day]['Upper']>2.5 and data[day]['Upper']<=3.5):
            fuzzy_upper['NULL']=0
            fuzzy_upper['SHORT']=0
            fuzzy_upper['MIDDLE']=1
            fuzzy_upper['LONG']=0

            Fuzzy_Upper='MIDDLE'

        elif(data[day]['Upper']>3.5 and data[day]['Upper']<=5):
            fuzzy_upper['NULL']=0
            fuzzy_upper['SHORT']=0
            fuzzy_upper['MIDDLE']=(-2*data[day]['Upper']+10)/3
            fuzzy_upper['LONG']=(2*data[day]['Upper']-7)/3

            if(max(fuzzy_upper['MIDDLE'], fuzzy_upper['LONG'])==fuzzy_upper['LONG']):
                Fuzzy_Upper='LONG'
            else:
                Fuzzy_Upper='MIDDLE'

        elif(data[day]['Upper']>5):
            fuzzy_upper['NULL']=0
            fuzzy_upper['SHORT']=0
            fuzzy_upper['MIDDLE']=0
            fuzzy_upper['LONG']=1

            Fuzzy_Upper='LONG'

        #body
        fuzzy_body={}
        if(data[day]['Body']<=-5):
            fuzzy_body['BLACK_LONG']=1
            fuzzy_body['BLACK_MIDDLE']=0
            fuzzy_body['BLACK_SHORT']=0
            fuzzy_body['EQUAL']=0
            fuzzy_body['WHITE_SHORT']=0
            fuzzy_body['WHITE_MIDDLE']=0
            fuzzy_body['WHITE_LONG']=0

            if(max(fuzzy_body['BLACK_LONG'],
            fuzzy_body['BLACK_MIDDLE'],
            fuzzy_body['BLACK_SHORT'],
            fuzzy_body['EQUAL'],
            fuzzy_body['WHITE_SHORT'],
            fuzzy_body['WHITE_MIDDLE'],
            fuzzy_body['WHITE_LONG'])==fuzzy_body['BLACK_LONG']):
                Fuzzy_Body='BLACK_LONG'
            
        if(data[day]['Body']>-5 and data[day]['Body']<=-3.5):
            fuzzy_body['BLACK_LONG']=(-2*data[day]['Body']-7)/3
            fuzzy_body['BLACK_MIDDLE']=(2*data[day]['Body']+10)/3
            fuzzy_body['BLACK_SHORT']=0
            fuzzy_body['EQUAL']=0
            fuzzy_body['WHITE_SHORT']=0
            fuzzy_body['WHITE_MIDDLE']=0
            fuzzy_body['WHITE_LONG']=0

            if(max(fuzzy_body['BLACK_LONG'],
            fuzzy_body['BLACK_MIDDLE'],
            fuzzy_body['BLACK_SHORT'],
            fuzzy_body['EQUAL'],
            fuzzy_body['WHITE_SHORT'],
            fuzzy_body['WHITE_MIDDLE'],
            fuzzy_body['WHITE_LONG'])==fuzzy_body['BLACK_LONG']):
                Fuzzy_Body='BLACK_LONG'
            
            else:
                Fuzzy_Body='BLACK_MIDDLE'
        
        if(data[day]['Body']>-3.5 and data[day]['Body']<=-2.5):
            fuzzy_body['BLACK_LONG']=0
            fuzzy_body['BLACK_MIDDLE']=1
            fuzzy_body['BLACK_SHORT']=0
            fuzzy_body['EQUAL']=0
            fuzzy_body['WHITE_SHORT']=0
            fuzzy_body['WHITE_MIDDLE']=0
            fuzzy_body['WHITE_LONG']=0

            Fuzzy_Body='BLACK_MIDDLE'

        if(data[day]['Body']>-2.5 and data[day]['Body']<=-1.5):
            fuzzy_body['BLACK_LONG']=0
            fuzzy_body['BLACK_MIDDLE']=-data[day]['Body']-1.5
            fuzzy_body['BLACK_SHORT']=data[day]['Body']+2.5
            fuzzy_body['EQUAL']=0
            fuzzy_body['WHITE_SHORT']=0
            fuzzy_body['WHITE_MIDDLE']=0
            fuzzy_body['WHITE_LONG']=0

            if(max(fuzzy_body['BLACK_LONG'],
            fuzzy_body['BLACK_MIDDLE'],
            fuzzy_body['BLACK_SHORT'],
            fuzzy_body['EQUAL'],
            fuzzy_body['WHITE_SHORT'],
            fuzzy_body['WHITE_MIDDLE'],
            fuzzy_body['WHITE_LONG'])==fuzzy_body['BLACK_MIDDLE']):
                Fuzzy_Body='BLACK_MIDDLE'
            
            else:
                Fuzzy_Body='BLACK_SHORT'
        
        if(data[day]['Body']>-1.5 and data[day]['Body']<=-0.5):
            fuzzy_body['BLACK_LONG']=0
            fuzzy_body['BLACK_MIDDLE']=0
            fuzzy_body['BLACK_SHORT']=1
            fuzzy_body['EQUAL']=0
            fuzzy_body['WHITE_SHORT']=0
            fuzzy_body['WHITE_MIDDLE']=0
            fuzzy_body['WHITE_LONG']=0

            Fuzzy_Body='BLACK_SHORT'

        if(data[day]['Body']>-0.5 and data[day]['Body']<=0):
            fuzzy_body['BLACK_LONG']=0
            fuzzy_body['BLACK_MIDDLE']=0
            fuzzy_body['BLACK_SHORT']=-2*data[day]['Body']
            fuzzy_body['EQUAL']=2*data[day]['Body']
            fuzzy_body['WHITE_SHORT']=0
            fuzzy_body['WHITE_MIDDLE']=0
            fuzzy_body['WHITE_LONG']=0

            if(max(fuzzy_body['BLACK_LONG'],
            fuzzy_body['BLACK_MIDDLE'],
            fuzzy_body['BLACK_SHORT'],
            fuzzy_body['EQUAL'],
            fuzzy_body['WHITE_SHORT'],
            fuzzy_body['WHITE_MIDDLE'],
            fuzzy_body['WHITE_LONG'])==fuzzy_body['BLACK_SHORT']):
                Fuzzy_Body='BLACK_SHORT'
            
            else:
                Fuzzy_Body='EQUAL'

        if(data[day]['Body']>0 and data[day]['Body']<=0.5):
            fuzzy_body['BLACK_LONG']=0
            fuzzy_body['BLACK_MIDDLE']=0
            fuzzy_body['BLACK_SHORT']=0
            fuzzy_body['EQUAL']=-2*data[day]['Body']+1
            fuzzy_body['WHITE_SHORT']=2*data[day]['Body']
            fuzzy_body['WHITE_MIDDLE']=0
            fuzzy_body['WHITE_LONG']=0

            if(max(fuzzy_body['BLACK_LONG'],
            fuzzy_body['BLACK_MIDDLE'],
            fuzzy_body['BLACK_SHORT'],
            fuzzy_body['EQUAL'],
            fuzzy_body['WHITE_SHORT'],
            fuzzy_body['WHITE_MIDDLE'],
            fuzzy_body['WHITE_LONG'])==fuzzy_body['EQUAL']):
                Fuzzy_Body='EQUAL'
            
            else:
                Fuzzy_Body='WHITE_SHORT'

        if(data[day]['Body']>0.5 and data[day]['Body']<=1.5):
            fuzzy_body['BLACK_LONG']=0
            fuzzy_body['BLACK_MIDDLE']=0
            fuzzy_body['BLACK_SHORT']=0
            fuzzy_body['EQUAL']=0
            fuzzy_body['WHITE_SHORT']=1
            fuzzy_body['WHITE_MIDDLE']=0
            fuzzy_body['WHITE_LONG']=0

            Fuzzy_Body='WHITE_SHORT'

        if(data[day]['Body']>1.5 and data[day]['Body']<=2.5):
            fuzzy_body['BLACK_LONG']=0
            fuzzy_body['BLACK_MIDDLE']=0
            fuzzy_body['BLACK_SHORT']=0
            fuzzy_body['EQUAL']=0
            fuzzy_body['WHITE_SHORT']=-data[day]['Body'] + 2.5
            fuzzy_body['WHITE_MIDDLE']=data[day]['Body'] - 1.5
            fuzzy_body['WHITE_LONG']=0

            if(max(fuzzy_body['BLACK_LONG'],
            fuzzy_body['BLACK_MIDDLE'],
            fuzzy_body['BLACK_SHORT'],
            fuzzy_body['EQUAL'],
            fuzzy_body['WHITE_SHORT'],
            fuzzy_body['WHITE_MIDDLE'],
            fuzzy_body['WHITE_LONG'])==fuzzy_body['WHITE_SHORT']):
                Fuzzy_Body='WHITE_SHORT'
            
            else:
                Fuzzy_Body='WHITE_MIDDLE'            

        if(data[day]['Body']>2.5 and data[day]['Body']<=3.5):
            fuzzy_body['BLACK_LONG']=0
            fuzzy_body['BLACK_MIDDLE']=0
            fuzzy_body['BLACK_SHORT']=0
            fuzzy_body['EQUAL']=0
            fuzzy_body['WHITE_SHORT']=0
            fuzzy_body['WHITE_MIDDLE']=1
            fuzzy_body['WHITE_LONG']=0

            Fuzzy_Body='WHITE_MIDDLE' 

        if(data[day]['Body']>3.5 and data[day]['Body']<=5):
            fuzzy_body['BLACK_LONG']=0
            fuzzy_body['BLACK_MIDDLE']=0
            fuzzy_body['BLACK_SHORT']=0
            fuzzy_body['EQUAL']=0
            fuzzy_body['WHITE_SHORT']=0
            fuzzy_body['WHITE_MIDDLE']=(-2*data[day]['Body']+10)/3
            fuzzy_body['WHITE_LONG']=(2*data[day]['Body']-7)/3

            if(max(fuzzy_body['BLACK_LONG'],
            fuzzy_body['BLACK_MIDDLE'],
            fuzzy_body['BLACK_SHORT'],
            fuzzy_body['EQUAL'],
            fuzzy_body['WHITE_SHORT'],
            fuzzy_body['WHITE_MIDDLE'],
            fuzzy_body['WHITE_LONG'])==fuzzy_body['WHITE_MIDDLE']):
                Fuzzy_Body='WHITE_MIDDLE'
            
            else:
                Fuzzy_Body='WHITE_LONG' 

        if(data[day]['Body']>5):
            fuzzy_body['BLACK_LONG']=0
            fuzzy_body['BLACK_MIDDLE']=0
            fuzzy_body['BLACK_SHORT']=0
            fuzzy_body['EQUAL']=0
            fuzzy_body['WHITE_SHORT']=0
            fuzzy_body['WHITE_MIDDLE']=0
            fuzzy_body['WHITE_LONG']=1

            Fuzzy_Body='WHITE_LONG' 

        #trend
        fuzzy_trend={}

        if(data[day]['Trend']<=-5):
            fuzzy_trend['LONG_BEARISH']=1
            fuzzy_trend['MIDDLE_BEARISH']=0
            fuzzy_trend['SHORT_BEARISH']=0
            fuzzy_trend['NULL']=0
            fuzzy_trend['SHORT_BULLISH']=0
            fuzzy_trend['MIDDLE_BULLISH']=0
            fuzzy_trend['LONG_BULLISH']=0

            Fuzzy_Trend='LONG_BEARISH'          
            
        if(data[day]['Trend']>-5 and data[day]['Trend']<=-3.5):
            fuzzy_trend['LONG_BEARISH']=(-2*data[day]['Trend']-7)/3
            fuzzy_trend['MIDDLE_BEARISH']=(2*data[day]['Trend']+10)/3
            fuzzy_trend['SHORT_BEARISH']=0
            fuzzy_trend['NULL']=0
            fuzzy_trend['SHORT_BULLISH']=0
            fuzzy_trend['MIDDLE_BULLISH']=0
            fuzzy_trend['LONG_BULLISH']=0

            if(max(fuzzy_trend['LONG_BEARISH'],
            fuzzy_trend['MIDDLE_BEARISH'],
            fuzzy_trend['SHORT_BEARISH'],
            fuzzy_trend['NULL'],
            fuzzy_trend['SHORT_BULLISH'],
            fuzzy_trend['MIDDLE_BULLISH'],
            fuzzy_trend['LONG_BULLISH'])==fuzzy_trend['LONG_BEARISH']):
                Fuzzy_Trend='LONG_BEARISH'
            
            else:
                Fuzzy_Trend='MIDDLE_BEARISH'
        
        if(data[day]['Trend']>-3.5 and data[day]['Trend']<=-2.5):
            fuzzy_trend['LONG_BEARISH']=0
            fuzzy_trend['MIDDLE_BEARISH']=1
            fuzzy_trend['SHORT_BEARISH']=0
            fuzzy_trend['NULL']=0
            fuzzy_trend['SHORT_BULLISH']=0
            fuzzy_trend['MIDDLE_BULLISH']=0
            fuzzy_trend['LONG_BULLISH']=0

            Fuzzy_Trend='MIDDLE_BEARISH'

        if(data[day]['Trend']>-2.5 and data[day]['Trend']<=-1.5):
            fuzzy_trend['LONG_BEARISH']=0
            fuzzy_trend['MIDDLE_BEARISH']=-data[day]['Trend']-1.5
            fuzzy_trend['SHORT_BEARISH']=data[day]['Trend']+2.5
            fuzzy_trend['NULL']=0
            fuzzy_trend['SHORT_BULLISH']=0
            fuzzy_trend['MIDDLE_BULLISH']=0
            fuzzy_trend['LONG_BULLISH']=0

            if(max(fuzzy_trend['LONG_BEARISH'],
            fuzzy_trend['MIDDLE_BEARISH'],
            fuzzy_trend['SHORT_BEARISH'],
            fuzzy_trend['NULL'],
            fuzzy_trend['SHORT_BULLISH'],
            fuzzy_trend['MIDDLE_BULLISH'],
            fuzzy_trend['LONG_BULLISH'])==fuzzy_trend['MIDDLE_BEARISH']):
                Fuzzy_Trend='MIDDLE_BEARISH'
            
            else:
                Fuzzy_Trend='SHORT_BEARISH'
        
        if(data[day]['Trend']>-1.5 and data[day]['Trend']<=-0.5):
            fuzzy_trend['LONG_BEARISH']=0
            fuzzy_trend['MIDDLE_BEARISH']=0
            fuzzy_trend['SHORT_BEARISH']=1
            fuzzy_trend['NULL']=0
            fuzzy_trend['SHORT_BULLISH']=0
            fuzzy_trend['MIDDLE_BULLISH']=0
            fuzzy_trend['LONG_BULLISH']=0

            Fuzzy_Trend='SHORT_BEARISH'

        if(data[day]['Trend']>-0.5 and data[day]['Trend']<=0):
            fuzzy_trend['LONG_BEARISH']=0
            fuzzy_trend['MIDDLE_BEARISH']=0
            fuzzy_trend['SHORT_BEARISH']=-2*data[day]['Trend']
            fuzzy_trend['NULL']=2*data[day]['Trend']
            fuzzy_trend['SHORT_BULLISH']=0
            fuzzy_trend['MIDDLE_BULLISH']=0
            fuzzy_trend['LONG_BULLISH']=0

            if(max(fuzzy_trend['LONG_BEARISH'],
            fuzzy_trend['MIDDLE_BEARISH'],
            fuzzy_trend['SHORT_BEARISH'],
            fuzzy_trend['NULL'],
            fuzzy_trend['SHORT_BULLISH'],
            fuzzy_trend['MIDDLE_BULLISH'],
            fuzzy_trend['LONG_BULLISH'])==fuzzy_trend['SHORT_BEARISH']):
                Fuzzy_Trend='SHORT_BEARISH'
            
            else:
                Fuzzy_Trend='NULL'

        if(data[day]['Trend']>0 and data[day]['Trend']<=0.5):
            fuzzy_trend['LONG_BEARISH']=0
            fuzzy_trend['MIDDLE_BEARISH']=0
            fuzzy_trend['SHORT_BEARISH']=0
            fuzzy_trend['NULL']=-2*data[day]['Trend']+1
            fuzzy_trend['SHORT_BULLISH']=2*data[day]['Trend']
            fuzzy_trend['MIDDLE_BULLISH']=0
            fuzzy_trend['LONG_BULLISH']=0

            if(max(fuzzy_trend['LONG_BEARISH'],
            fuzzy_trend['MIDDLE_BEARISH'],
            fuzzy_trend['SHORT_BEARISH'],
            fuzzy_trend['NULL'],
            fuzzy_trend['SHORT_BULLISH'],
            fuzzy_trend['MIDDLE_BULLISH'],
            fuzzy_trend['LONG_BULLISH'])==fuzzy_trend['SHORT_BULLISH']):
                Fuzzy_Trend='SHORT_BULLISH'
            
            else:
                Fuzzy_Trend='NULL'

        if(data[day]['Trend']>0.5 and data[day]['Trend']<=1.5):
            fuzzy_trend['LONG_BEARISH']=0
            fuzzy_trend['MIDDLE_BEARISH']=0
            fuzzy_trend['SHORT_BEARISH']=0
            fuzzy_trend['NULL']=0
            fuzzy_trend['SHORT_BULLISH']=1
            fuzzy_trend['MIDDLE_BULLISH']=0
            fuzzy_trend['LONG_BULLISH']=0

            Fuzzy_Trend='SHORT_BULLISH'

        if(data[day]['Trend']>1.5 and data[day]['Trend']<=2.5):
            fuzzy_trend['LONG_BEARISH']=0
            fuzzy_trend['MIDDLE_BEARISH']=0
            fuzzy_trend['SHORT_BEARISH']=0
            fuzzy_trend['NULL']=0
            fuzzy_trend['SHORT_BULLISH']=-data[day]['Trend'] + 2.5
            fuzzy_trend['MIDDLE_BULLISH']=data[day]['Trend'] - 1.5
            fuzzy_trend['LONG_BULLISH']=0

            if(max(fuzzy_trend['LONG_BEARISH'],
            fuzzy_trend['MIDDLE_BEARISH'],
            fuzzy_trend['SHORT_BEARISH'],
            fuzzy_trend['NULL'],
            fuzzy_trend['SHORT_BULLISH'],
            fuzzy_trend['MIDDLE_BULLISH'],
            fuzzy_trend['LONG_BULLISH'])==fuzzy_trend['SHORT_BULLISH']):
                Fuzzy_Trend='SHORT_BULLISH'
            
            else:
                Fuzzy_Trend='MIDDLE_BULLISH'

        if(data[day]['Trend']>2.5 and data[day]['Trend']<=3.5):
            fuzzy_trend['LONG_BEARISH']=0
            fuzzy_trend['MIDDLE_BEARISH']=0
            fuzzy_trend['SHORT_BEARISH']=0
            fuzzy_trend['NULL']=0
            fuzzy_trend['SHORT_BULLISH']=0
            fuzzy_trend['MIDDLE_BULLISH']=1
            fuzzy_trend['LONG_BULLISH']=0

            Fuzzy_Trend='MIDDLE_BULLISH'

        if(data[day]['Trend']>3.5 and data[day]['Trend']<=5):
            fuzzy_trend['LONG_BEARISH']=0
            fuzzy_trend['MIDDLE_BEARISH']=0
            fuzzy_trend['SHORT_BEARISH']=0
            fuzzy_trend['NULL']=0
            fuzzy_trend['SHORT_BULLISH']=0
            fuzzy_trend['MIDDLE_BULLISH']=(-2*data[day]['Trend']+10)/3
            fuzzy_trend['LONG_BULLISH']=(2*data[day]['Trend']-7)/3

            if(max(fuzzy_trend['LONG_BEARISH'],
            fuzzy_trend['MIDDLE_BEARISH'],
            fuzzy_trend['SHORT_BEARISH'],
            fuzzy_trend['NULL'],
            fuzzy_trend['SHORT_BULLISH'],
            fuzzy_trend['MIDDLE_BULLISH'],
            fuzzy_trend['LONG_BULLISH'])==fuzzy_trend['LONG_BULLISH']):
                Fuzzy_Trend='LONG_BULLISH'
            
            else:
                Fuzzy_Trend='MIDDLE_BULLISH'

        if(data[day]['Trend']>5):
            fuzzy_trend['LONG_BEARISH']=0
            fuzzy_trend['MIDDLE_BEARISH']=0
            fuzzy_trend['SHORT_BEARISH']=0
            fuzzy_trend['NULL']=0
            fuzzy_trend['SHORT_BULLISH']=0
            fuzzy_trend['MIDDLE_BULLISH']=0
            fuzzy_trend['LONG_BULLISH']=1

            Fuzzy_Trend='LONG_BULLISH'


        #gap
        fuzzy_gap={}

        if(data[day]['Gap']>=0 and data[day]['Gap']<=0.5):
            fuzzy_gap['NULL']=-2*data[day]['Gap']+1
            fuzzy_gap['SHORT']=2*data[day]['Gap']
            fuzzy_gap['MIDDLE']=0
            fuzzy_gap['LONG']=0

            if(max(fuzzy_gap['NULL'],
            fuzzy_gap['SHORT'],
            fuzzy_gap['MIDDLE'],
            fuzzy_gap['LONG'])==fuzzy_gap['NULL']):
                Fuzzy_Gap='NULL'
            
            else:
                Fuzzy_Gap='SHORT'

        if(data[day]['Gap']>0.5 and data[day]['Gap']<=1):
            fuzzy_gap['NULL']=0
            fuzzy_gap['SHORT']=1
            fuzzy_gap['MIDDLE']=0
            fuzzy_gap['LONG']=0

            Fuzzy_Gap='SHORT'

        if(data[day]['Gap']>1 and data[day]['Gap']<=1.5):
            fuzzy_gap['NULL']=0
            fuzzy_gap['SHORT']=-2*data[day]['Gap']+3
            fuzzy_gap['MIDDLE']=2*data[day]['Gap']-2
            fuzzy_gap['LONG']=0

            if(max(fuzzy_gap['NULL'],
            fuzzy_gap['SHORT'],
            fuzzy_gap['MIDDLE'],
            fuzzy_gap['LONG'])==fuzzy_gap['MIDDLE']):
                Fuzzy_Gap='MIDDLE'
            
            else:
                Fuzzy_Gap='SHORT'

        if(data[day]['Gap']>1.5 and data[day]['Gap']<=2):
            fuzzy_gap['NULL']=0
            fuzzy_gap['SHORT']=0
            fuzzy_gap['MIDDLE']=1
            fuzzy_gap['LONG']=0

            Fuzzy_Gap='MIDDLE'

        if(data[day]['Gap']>2 and data[day]['Gap']<=2.5):
            fuzzy_gap['NULL']=0
            fuzzy_gap['SHORT']=0
            fuzzy_gap['MIDDLE']=-2*data[day]['Gap']+5
            fuzzy_gap['LONG']=2*data[day]['Gap']-4

            if(max(fuzzy_gap['NULL'],
            fuzzy_gap['SHORT'],
            fuzzy_gap['MIDDLE'],
            fuzzy_gap['LONG'])==fuzzy_gap['MIDDLE']):
                Fuzzy_Gap='MIDDLE'
            
            else:
                Fuzzy_Gap='LONG'

        if(data[day]['Gap']>2.5):
            fuzzy_gap['NULL']=0
            fuzzy_gap['SHORT']=0
            fuzzy_gap['MIDDLE']=0
            fuzzy_gap['LONG']=1

            Fuzzy_Gap='LONG'

        #difopen

        fuzzy_difopen={}

        if(data[day]['Difopen']>=0 and data[day]['Difopen']<=0.5):
            fuzzy_difopen['NULL']=-2*data[day]['Difopen']+1
            fuzzy_difopen['SHORT']=2*data[day]['Difopen']
            fuzzy_difopen['MIDDLE']=0
            fuzzy_difopen['LONG']=0

            if(max(fuzzy_difopen['NULL'],
            fuzzy_difopen['SHORT'],
            fuzzy_difopen['MIDDLE'],
            fuzzy_difopen['LONG'])==fuzzy_difopen['NULL']):
                Fuzzy_Difopen='NULL'
            
            else:
                Fuzzy_Difopen='SHORT'

        if(data[day]['Difopen']>0.5 and data[day]['Difopen']<=1.5):
            fuzzy_difopen['NULL']=0
            fuzzy_difopen['SHORT']=1
            fuzzy_difopen['MIDDLE']=0
            fuzzy_difopen['LONG']=0

            Fuzzy_Difopen='SHORT'

        if(data[day]['Difopen']>1.5 and data[day]['Difopen']<=2.5):
            fuzzy_difopen['NULL']=0
            fuzzy_difopen['SHORT']=-data[day]['Difopen']+2.5
            fuzzy_difopen['MIDDLE']=data[day]['Difopen']-1.5
            fuzzy_difopen['LONG']=0

            if(max(fuzzy_difopen['NULL'],
            fuzzy_difopen['SHORT'],
            fuzzy_difopen['MIDDLE'],
            fuzzy_difopen['LONG'])==fuzzy_difopen['SHORT']):
                Fuzzy_Difopen='SHORT'
            
            else:
                Fuzzy_Difopen='MIDDLE'

        if(data[day]['Difopen']>2.5 and data[day]['Difopen']<=3.5):
            fuzzy_difopen['NULL']=0
            fuzzy_difopen['SHORT']=0
            fuzzy_difopen['MIDDLE']=1
            fuzzy_difopen['LONG']=0

            Fuzzy_Difopen='MIDDLE'

        if(data[day]['Difopen']>3.5 and data[day]['Difopen']<=5):
            fuzzy_difopen['NULL']=0
            fuzzy_difopen['SHORT']=0
            fuzzy_difopen['MIDDLE']=(-2*data[day]['Difopen']+10)/3
            fuzzy_difopen['LONG']=(2*data[day]['Difopen']-7)/3

            if(max(fuzzy_difopen['NULL'],
            fuzzy_difopen['SHORT'],
            fuzzy_difopen['MIDDLE'],
            fuzzy_difopen['LONG'])==fuzzy_difopen['LONG']):
                Fuzzy_Difopen='LONG'
            
            else:
                Fuzzy_Difopen='MIDDLE'

        if(data[day]['Difopen']>5):
            fuzzy_difopen['NULL']=0
            fuzzy_difopen['SHORT']=0
            fuzzy_difopen['MIDDLE']=0
            fuzzy_difopen['LONG']=1

            Fuzzy_Difopen='LONG'

        #difclose

        fuzzy_difclose={}

        if(data[day]['Difclose']>=0 and data[day]['Difclose']<=0.5):
            fuzzy_difclose['NULL']=-2*data[day]['Difclose']+1
            fuzzy_difclose['SHORT']=2*data[day]['Difclose']
            fuzzy_difclose['MIDDLE']=0
            fuzzy_difclose['LONG']=0

            if(max(fuzzy_difclose['NULL'],
            fuzzy_difclose['SHORT'],
            fuzzy_difclose['MIDDLE'],
            fuzzy_difclose['LONG'])==fuzzy_difclose['NULL']):
                Fuzzy_Difclose='NULL'
            
            else:
                Fuzzy_Difclose='SHORT'

        if(data[day]['Difclose']>0.5 and data[day]['Difclose']<=1.5):
            fuzzy_difclose['NULL']=0
            fuzzy_difclose['SHORT']=1
            fuzzy_difclose['MIDDLE']=0
            fuzzy_difclose['LONG']=0

            Fuzzy_Difclose='SHORT'

        if(data[day]['Difclose']>1.5 and data[day]['Difclose']<=2.5):
            fuzzy_difclose['NULL']=0
            fuzzy_difclose['SHORT']=-data[day]['Difclose']+2.5
            fuzzy_difclose['MIDDLE']=data[day]['Difclose']-1.5
            fuzzy_difclose['LONG']=0

            if(max(fuzzy_difclose['NULL'],
            fuzzy_difclose['SHORT'],
            fuzzy_difclose['MIDDLE'],
            fuzzy_difclose['LONG'])==fuzzy_difclose['SHORT']):
                Fuzzy_Difclose='SHORT'
            
            else:
                Fuzzy_Difclose='MIDDLE'

        if(data[day]['Difclose']>2.5 and data[day]['Difclose']<=3.5):
            fuzzy_difclose['NULL']=0
            fuzzy_difclose['SHORT']=0
            fuzzy_difclose['MIDDLE']=1
            fuzzy_difclose['LONG']=0

            Fuzzy_Difclose='MIDDLE'

        if(data[day]['Difclose']>3.5 and data[day]['Difclose']<=5):
            fuzzy_difclose['NULL']=0
            fuzzy_difclose['SHORT']=0
            fuzzy_difclose['MIDDLE']=(-2*data[day]['Difclose']+10)/3
            fuzzy_difclose['LONG']=(2*data[day]['Difclose']-7)/3

            if(max(fuzzy_difclose['NULL'],
            fuzzy_difclose['SHORT'],
            fuzzy_difclose['MIDDLE'],
            fuzzy_difclose['LONG'])==fuzzy_difclose['LONG']):
                Fuzzy_Difclose='LONG'
            
            else:
                Fuzzy_Difclose='MIDDLE'

        if(data[day]['Difclose']>5):
            fuzzy_difclose['NULL']=0
            fuzzy_difclose['SHORT']=0
            fuzzy_difclose['MIDDLE']=0
            fuzzy_difclose['LONG']=1

            Fuzzy_Difclose='LONG'
            
        #difcentral

        fuzzy_difcentral={}

        if(data[day]['Difcentral']>=0 and data[day]['Difcentral']<=0.5):
            fuzzy_difcentral['NULL']=-2*data[day]['Difcentral']+1
            fuzzy_difcentral['SHORT']=2*data[day]['Difcentral']
            fuzzy_difcentral['MIDDLE']=0
            fuzzy_difcentral['LONG']=0

            if(max(fuzzy_difcentral['NULL'],
            fuzzy_difcentral['SHORT'],
            fuzzy_difcentral['MIDDLE'],
            fuzzy_difcentral['LONG'])==fuzzy_difcentral['NULL']):
                Fuzzy_Difcentral='NULL'
            
            else:
                Fuzzy_Difcentral='SHORT'

        if(data[day]['Difcentral']>0.5 and data[day]['Difcentral']<=1.5):
            fuzzy_difcentral['NULL']=0
            fuzzy_difcentral['SHORT']=1
            fuzzy_difcentral['MIDDLE']=0
            fuzzy_difcentral['LONG']=0

            Fuzzy_Difcentral='SHORT'

        if(data[day]['Difcentral']>1.5 and data[day]['Difcentral']<=2.5):
            fuzzy_difcentral['NULL']=0
            fuzzy_difcentral['SHORT']=-data[day]['Difcentral']+2.5
            fuzzy_difcentral['MIDDLE']=data[day]['Difcentral']-1.5
            fuzzy_difcentral['LONG']=0

            if(max(fuzzy_difcentral['NULL'],
            fuzzy_difcentral['SHORT'],
            fuzzy_difcentral['MIDDLE'],
            fuzzy_difcentral['LONG'])==fuzzy_difcentral['MIDDLE']):
                Fuzzy_Difcentral='MIDDLE'
            
            else:
                Fuzzy_Difcentral='SHORT'

        if(data[day]['Difcentral']>2.5 and data[day]['Difcentral']<=3.5):
            fuzzy_difcentral['NULL']=0
            fuzzy_difcentral['SHORT']=0
            fuzzy_difcentral['MIDDLE']=1
            fuzzy_difcentral['LONG']=0

            Fuzzy_Difcentral='MIDDLE'

        if(data[day]['Difcentral']>3.5 and data[day]['Difcentral']<=5):
            fuzzy_difcentral['NULL']=0
            fuzzy_difcentral['SHORT']=0
            fuzzy_difcentral['MIDDLE']=(-2*data[day]['Difcentral']+10)/3
            fuzzy_difcentral['LONG']=(2*data[day]['Difcentral']-7)/3

            if(max(fuzzy_difcentral['NULL'],
            fuzzy_difcentral['SHORT'],
            fuzzy_difcentral['MIDDLE'],
            fuzzy_difcentral['LONG'])==fuzzy_difcentral['MIDDLE']):
                Fuzzy_Difcentral='MIDDLE'
            
            else:
                Fuzzy_Difcentral='LONG'

        if(data[day]['Difcentral']>5):
            fuzzy_difcentral['NULL']=0
            fuzzy_difcentral['SHORT']=0
            fuzzy_difcentral['MIDDLE']=0
            fuzzy_difcentral['LONG']=1

            Fuzzy_Difcentral='LONG'

        # RSI

        if(data[day]['RSI']<=55 and data[day]['RSI']>45):
            RSI='NULL'
        if(data[day]['RSI']<=65 and data[day]['RSI']>55):
            RSI='LOW_BULLISH'
        if(data[day]['RSI']<=75 and data[day]['RSI']>65):
            RSI='MEDIUM_BULLISH'
        if(data[day]['RSI']<=85 and data[day]['RSI']>75):
            RSI='HIGH_BULLISH'
        if(data[day]['RSI']<=100 and data[day]['RSI']>85):
            RSI='VERY_HIGH_BULLISH'
        if(data[day]['RSI']<=45 and data[day]['RSI']>35):
            RSI='LOW_BEARISH'
        if(data[day]['RSI']<=35 and data[day]['RSI']>25):
            RSI='MEDIUM_BEARISH'
        if(data[day]['RSI']<=25 and data[day]['RSI']>=0):
            RSI='HIGH_BEARISH'
        if(data[day]['RSI']<=85 and data[day]['RSI']>75):
            RSI='VERY_HIGH_BEARISH'
        
            
        fuzzified_candlestick_cluster[day]={'Fuzzy_Lower': Fuzzy_Lower, 'Fuzzy_Upper': Fuzzy_Upper, 'Fuzzy_Body': Fuzzy_Body, 'Fuzzy_Trend': Fuzzy_Trend, 'Fuzzy_Gap': Fuzzy_Gap, 'Fuzzy_Difopen': Fuzzy_Difopen, 'Fuzzy_Difclose': Fuzzy_Difclose,  'Fuzzy_Difcentral': Fuzzy_Difcentral, 'RSI': RSI}
    return fuzzified_candlestick_cluster

In [ ]:
data = pd.read_csv("query_data_with_rsi.csv")
f1=open('./query_doc2.txt', 'w+')
f2=open('./query_doc.txt', 'w+')
query=1

k = 1
datasheet = {}

for i,j in data.iterrows():
    d = { 'Open' : j.Open, 'High' : j.High, 'Low' : j.Low, 'Close' : j.Close, 'Body': abs(j.Open - j.Close), 'RSI': j.RSI }

    datasheet[k]=d
    k = k+1

y=6
x=0
clusters={}
for index in range(len(datasheet)-5):
    if(index==0):
        continue

    d={0 : datasheet[index], 
     1 : datasheet[index+1],
     2 : datasheet[index+2],
     3 : datasheet[index+3],
     4 : datasheet[index+4],
     5 : datasheet[index+5]}
    clusters[x] = d
    
    x+=1
count_of_identified_candlestick_cluster = {
'Kicking_Bullish' : 0,
    'Kicking_Bearish' : 0,
    'Engulfing_Bullish' : 0,
    'Engulfing_Bearish' : 0,
    'Harami_Bullish' : 0,
    'Harami_Bearish' : 0,
    'Meeting_Line_Bullish' : 0,
    'Meeting_Line_Bearish' : 0,
    'Hammer' : 0,
    'Inverted_Hammer' : 0,
    'Piercing_Line' : 0,
    'One_White_Soldier' : 0,
    'Homing_Pigeon' : 0,
    'Hanging_Man' : 0,
    'Descending_Hawk' : 0,
    'One_Black_Crow' : 0,
    'Dark_Cloud_Clover' : 0}


div_count=0
sr_count=0
data_for_future_trend={}
hew=0
max_hew=0

for index in range(len(clusters)-1):
    print("\n ", index)
    identified_candlestick_cluster=""
    
    cluster=clusters[index]    
    print("Cluster \n ", cluster)
    candlestick_cluster=candlestick(cluster)
    print("Candlestick Cluster \n ", candlestick_cluster)
    fuzzified_candlestick_cluster=fuzzify_candlestick(candlestick_cluster)
    print("Fuzzified Candlestick Cluster \n ", fuzzified_candlestick_cluster)



  0
Cluster 
  {0: {'Open': 37025.27, 'High': 37104.79, 'Low': 36610.57, 'Close': 36690.5, 'Body': 334.7699999999968, 'RSI': 0.0}, 1: {'Open': 36808.01, 'High': 37405.48, 'Low': 36655.41, 'Close': 37327.36, 'Body': 519.3499999999985, 'RSI': 0.0}, 2: {'Open': 37521.3, 'High': 37807.55, 'Low': 37406.26, 'Close': 37581.91, 'Body': 60.61000000000058, 'RSI': 0.0}, 3: {'Open': 37755.16, 'High': 37755.16, 'Low': 36888.49, 'Close': 36958.16, 'Body': 797.0, 'RSI': 0.0}, 4: {'Open': 37233.5, 'High': 37473.61, 'Low': 37000.77, 'Close': 37311.53, 'Body': 78.02999999999884, 'RSI': 0.0}, 5: {'Open': 37383.0, 'High': 37444.45, 'Low': 36974.41, 'Close': 37350.33, 'Body': 32.669999999998254, 'RSI': 67.2972165278164}}
Candlestick Cluster 
  {1: {'Upper': 0.21223641267214016, 'Lower': 0.414583673499324, 'Body': 1.3913386856182663, 'Gap': 0, 'Trend': 1.7061479836773898, 'Difopen': 0, 'Difclose': 0.5962650452643844, 'Difcentral': 1.2577235571977385, 'RSI': 0.0}, 2: {'Upper': 0.6013650912948096, 'Lower': 0